In [132]:
from utils.BackTest import Engine, Strategy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [133]:
df_1 = pd.read_csv('../data/processed/merged_df.csv')
df_2 = pd.read_csv('../data/processed/features_trading_strategy.csv')

In [134]:
df_1

,datetime,EMA_8,EMA_13,EMA_21,EMA_signal,EMA_short,EMA_long,MACD,Signal_Line,MACD_Hist,...,OBV_Signal,ATR,reddit_smoothed_sentiment_weibull,return,hourly_volatility,TOTAL_TRADES,VOLUME,smoothed_sentiment,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
0,2024-03-02 02:00:00+00:00,62220.598138,62172.294796,62076.132135,1,62184.157168,62011.323303,172.833864,187.562435,-14.728570,...,1,384.756667,0.157746,0.001309,0.006053,56088,1153.46042,-222.074588,-188.470966,0.000000
1,2024-03-02 03:00:00+00:00,62239.898552,62191.602682,62097.161032,1,62203.125296,62033.258614,169.866682,184.023284,-14.156603,...,0,347.451667,0.092267,0.003160,0.004931,67185,1465.47878,-204.266170,-204.409405,0.552411
2,2024-03-02 04:00:00+00:00,62194.576652,62169.366585,62091.596392,1,62177.406020,62033.457976,143.948043,176.008236,-32.060193,...,-1,333.268333,0.050600,-0.004357,0.005180,56098,1147.71004,-234.015147,-205.344729,0.000000
3,2024-03-02 05:00:00+00:00,62152.792951,62146.107073,62083.864902,1,62151.120478,62031.464793,119.655685,164.737726,-45.082040,...,0,326.113333,0.026060,-0.000474,0.004688,65329,1415.65823,-225.408115,-200.706776,0.000000
4,2024-03-02 06:00:00+00:00,62106.838962,62117.520348,62071.331729,0,62119.563482,62025.134067,94.429414,150.676063,-56.246649,...,0,325.211667,0.012619,-0.000977,0.004189,57826,1059.53378,-208.216550,-185.091972,1.010990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,2025-01-26 07:00:00+00:00,104968.753394,104930.387351,104892.740104,1,104936.968508,104874.604967,62.363541,41.106721,21.256820,...,1,298.715000,0.055311,0.000072,0.001646,41775,208.35672,30.142278,-13.564028,0.000000
8377,2025-01-26 08:00:00+00:00,104982.452640,104944.674872,104905.254640,1,104951.342584,104886.145340,65.197244,45.924826,19.272419,...,0,325.368333,0.030398,0.000305,0.001628,63281,283.50224,35.082025,-6.781369,0.103237
8378,2025-01-26 09:00:00+00:00,104904.136498,104899.725605,104880.234218,1,104901.909879,104867.173833,34.736046,43.687070,-8.951024,...,-1,326.935000,0.015620,-0.003812,0.001915,91103,621.23723,37.058349,0.681248,0.000000
8379,2025-01-26 10:00:00+00:00,104862.170609,104873.377661,104865.239289,-1,104873.199128,104855.923179,17.275949,38.404846,-21.128896,...,1,277.221667,0.007520,0.000815,0.001946,50517,252.60356,33.197390,8.190720,0.000000


In [135]:
df_1 = df_1.drop_duplicates(subset=['datetime'], keep='first')

In [136]:
df_1

,datetime,EMA_8,EMA_13,EMA_21,EMA_signal,EMA_short,EMA_long,MACD,Signal_Line,MACD_Hist,...,OBV_Signal,ATR,reddit_smoothed_sentiment_weibull,return,hourly_volatility,TOTAL_TRADES,VOLUME,smoothed_sentiment,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
0,2024-03-02 02:00:00+00:00,62220.598138,62172.294796,62076.132135,1,62184.157168,62011.323303,172.833864,187.562435,-14.728570,...,1,384.756667,0.157746,0.001309,0.006053,56088,1153.46042,-222.074588,-188.470966,0.000000
1,2024-03-02 03:00:00+00:00,62239.898552,62191.602682,62097.161032,1,62203.125296,62033.258614,169.866682,184.023284,-14.156603,...,0,347.451667,0.092267,0.003160,0.004931,67185,1465.47878,-204.266170,-204.409405,0.552411
2,2024-03-02 04:00:00+00:00,62194.576652,62169.366585,62091.596392,1,62177.406020,62033.457976,143.948043,176.008236,-32.060193,...,-1,333.268333,0.050600,-0.004357,0.005180,56098,1147.71004,-234.015147,-205.344729,0.000000
3,2024-03-02 05:00:00+00:00,62152.792951,62146.107073,62083.864902,1,62151.120478,62031.464793,119.655685,164.737726,-45.082040,...,0,326.113333,0.026060,-0.000474,0.004688,65329,1415.65823,-225.408115,-200.706776,0.000000
4,2024-03-02 06:00:00+00:00,62106.838962,62117.520348,62071.331729,0,62119.563482,62025.134067,94.429414,150.676063,-56.246649,...,0,325.211667,0.012619,-0.000977,0.004189,57826,1059.53378,-208.216550,-185.091972,1.010990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,2025-01-26 07:00:00+00:00,104968.753394,104930.387351,104892.740104,1,104936.968508,104874.604967,62.363541,41.106721,21.256820,...,1,298.715000,0.055311,0.000072,0.001646,41775,208.35672,30.142278,-13.564028,0.000000
8377,2025-01-26 08:00:00+00:00,104982.452640,104944.674872,104905.254640,1,104951.342584,104886.145340,65.197244,45.924826,19.272419,...,0,325.368333,0.030398,0.000305,0.001628,63281,283.50224,35.082025,-6.781369,0.103237
8378,2025-01-26 09:00:00+00:00,104904.136498,104899.725605,104880.234218,1,104901.909879,104867.173833,34.736046,43.687070,-8.951024,...,-1,326.935000,0.015620,-0.003812,0.001915,91103,621.23723,37.058349,0.681248,0.000000
8379,2025-01-26 10:00:00+00:00,104862.170609,104873.377661,104865.239289,-1,104873.199128,104855.923179,17.275949,38.404846,-21.128896,...,1,277.221667,0.007520,0.000815,0.001946,50517,252.60356,33.197390,8.190720,0.000000


In [137]:
df_2

,datetime,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,VOLUME,VOLUME_BUY,...,%D,Stochastic_Signal,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR
0,2024-02-22 19:00:00+00:00,51718.81,51718.82,51460.80,51509.99,50652,27901,22751,1412.76654,602.38295,...,NaN,0,NaN,NaN,0,258.02,NaN,NaN,258.02,NaN
1,2024-02-22 20:00:00+00:00,51509.99,52065.78,51461.10,51995.58,108926,71626,37300,3121.92839,1779.83133,...,NaN,0,3121.92839,3121.928390,0,604.68,555.79,48.89,604.68,NaN
2,2024-02-22 21:00:00+00:00,51995.58,52007.65,51545.26,51656.96,54274,25981,28293,1264.97024,667.58788,...,NaN,0,-1264.97024,2704.128520,-1,462.39,12.07,450.32,462.39,NaN
3,2024-02-22 22:00:00+00:00,51656.96,51717.87,51184.04,51456.01,73657,24349,49308,1357.06045,695.54171,...,NaN,0,-1357.06045,2317.348619,0,533.83,60.91,472.92,533.83,NaN
4,2024-02-22 23:00:00+00:00,51456.01,51482.69,51230.00,51288.42,38825,16468,22357,1038.76395,535.06306,...,NaN,0,-1038.76395,1997.718850,0,252.69,26.68,226.01,252.69,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2025-03-03 14:00:00+00:00,93586.83,93721.37,89136.45,89278.88,909864,449798,460066,6172.59027,2600.65274,...,51.833151,0,-6172.59027,91.667872,-1,4584.92,134.54,4450.38,4584.92,1402.496667
8996,2025-03-03 15:00:00+00:00,89278.88,91307.00,89117.00,90035.37,815439,415993,399446,5771.66863,2921.20810,...,39.186775,0,5771.66863,632.620325,1,2190.00,2028.12,161.88,2190.00,1618.750000
8997,2025-03-03 16:00:00+00:00,90035.37,90750.91,89691.02,90154.00,448660,224630,224030,2111.23659,1038.58417,...,12.624798,0,2111.23659,773.440922,0,1059.89,715.54,344.35,1059.89,1622.873333
8998,2025-03-03 17:00:00+00:00,90154.00,91000.00,89912.00,90300.01,331538,185179,146359,1626.75033,767.44993,...,20.525927,0,1626.75033,854.708485,0,1088.00,846.00,242.00,1088.00,1720.470000


In [138]:
df_3 = df_1[['datetime','reddit_smoothed_sentiment_weibull','news_smoothed_sentiment_weibull','tele_smoothed_sentiment_weibull']]

In [139]:
df_3[1:].describe()

,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
count,7.929000e+03,7929.000000,7929.000000
mean,2.994162e-02,-105.388053,0.304112
std,8.777959e-01,395.040101,0.822927
min,-4.277553e+00,-5128.788745,-3.930538
25%,-3.054081e-01,-257.068523,0.000000
50%,-9.775172e-18,-70.549400,0.000000
75%,2.992846e-01,99.450326,0.000000
max,5.323996e+00,1046.597034,7.955905


In [140]:
df_4 = pd.merge(df_3, df_2, on='datetime', how='inner')

In [141]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7930 entries, 0 to 7929
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   datetime                           7930 non-null   object 
 1   reddit_smoothed_sentiment_weibull  7930 non-null   float64
 2   news_smoothed_sentiment_weibull    7930 non-null   float64
 3   tele_smoothed_sentiment_weibull    7930 non-null   float64
 4   OPEN                               7930 non-null   float64
 5   HIGH                               7930 non-null   float64
 6   LOW                                7930 non-null   float64
 7   CLOSE                              7930 non-null   float64
 8   TOTAL_TRADES                       7930 non-null   int64  
 9   TOTAL_TRADES_BUY                   7930 non-null   int64  
 10  TOTAL_TRADES_SELL                  7930 non-null   int64  
 11  VOLUME                             7930 non-null   float

In [142]:
def custom_minmax_zero_anchor(series):
    x = series.copy()
    max_pos = x[x > 0].max()
    min_neg = x[x < 0].min()

    # Scale positive values
    if max_pos != 0:
        x[x > 0] = x[x > 0] / max_pos
    else:
        x[x > 0] = 0

    # Scale negative values
    if min_neg != 0:
        neg_scaled = x[x < 0] / abs(min_neg)
        x[x < 0] = neg_scaled  # values are negative, because x[x < 0] and min_neg are both negative
    else:
        x[x < 0] = 0

    return x

In [143]:
cols = [
    'reddit_smoothed_sentiment_weibull',
    'news_smoothed_sentiment_weibull',
    'tele_smoothed_sentiment_weibull'
]

# Apply to each column (column-wise)
df_4_scaled = df_4.copy()

# Normalize each column individually
normalized = df_4[cols].apply(custom_minmax_zero_anchor)

# Assign normalized columns with new names
normalized.columns = [col + '_normalized' for col in cols]
df_4_scaled = pd.concat([df_4_scaled, normalized], axis=1)

In [144]:
df_4_scaled

,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,...,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR,reddit_smoothed_sentiment_weibull_normalized,news_smoothed_sentiment_weibull_normalized,tele_smoothed_sentiment_weibull_normalized
0,2024-03-02 02:00:00+00:00,0.157746,-188.470966,0.000000,62030.01,62216.71,61949.03,62111.18,56088,24390,...,-263.362181,1,267.68,186.70,80.98,267.68,384.756667,0.029629,-0.036748,0.000000
1,2024-03-02 03:00:00+00:00,0.092267,-204.409405,0.552411,62111.18,62426.70,62025.00,62307.45,67185,27772,...,-98.710661,0,401.70,315.52,86.18,401.70,347.451667,0.017330,-0.039855,0.069434
2,2024-03-02 04:00:00+00:00,0.050600,-205.344729,0.000000,62307.45,62307.45,62035.95,62035.95,56098,24794,...,-198.615364,-1,271.50,0.00,271.50,271.50,333.268333,0.009504,-0.040038,0.000000
3,2024-03-02 05:00:00+00:00,0.026060,-200.706776,0.000000,62035.95,62079.99,61752.30,62006.55,65329,23953,...,-314.524208,0,327.69,44.04,283.65,327.69,326.113333,0.004895,-0.039133,0.000000
4,2024-03-02 06:00:00+00:00,0.012619,-185.091972,1.010990,62006.55,62256.00,61930.00,61946.00,57826,25109,...,-385.477501,0,326.00,249.45,76.55,326.00,325.211667,0.002370,-0.036089,0.127074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,22248,...,63.377330,1,150.00,109.25,40.75,150.00,298.715000,0.010389,-0.002645,0.000000
7926,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,30294,...,84.341607,0,358.05,223.01,135.04,358.05,325.368333,0.005710,-0.001322,0.012976
7927,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,41647,...,17.143622,-1,586.40,0.01,586.39,586.40,326.935000,0.002934,0.000651,0.000000
7928,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,26960,...,39.568378,1,222.90,92.87,130.03,222.90,277.221667,0.001412,0.007826,0.000000


In [145]:
# # Specify target range
# scaler = MinMaxScaler(feature_range=(-1, 1))

# df_4[['reddit_smoothed_sentiment_weibull_normalized','news_smoothed_sentiment_weibull_normalized','tele_smoothed_sentiment_weibull_normalized']] = scaler.fit_transform(df_4[['reddit_smoothed_sentiment_weibull','news_smoothed_sentiment_weibull','tele_smoothed_sentiment_weibull']])

In [146]:
#VWAP strategy
def determine_signal(df):
    """
    Determine the trading signal based on the strategy's criteria.
    """
    if df['CLOSE'] > df['VWAP']:
        return 1
    elif df['CLOSE'] < df['VWAP']:
        return 0
    else:
        return None

def calculate_vwap(df):
    """
    Calculate the VWAP (Volume Weighted Average Price) for each bar in the data.
    """
    df['VWAP'] = (df['CLOSE'] * df['VOLUME']).cumsum() / df['VOLUME'].cumsum()



In [147]:
df_4_scaled['VWAP'] = (df_4_scaled['CLOSE'] * df_4_scaled['VOLUME']).cumsum() / df_4_scaled['VOLUME'].cumsum()

In [148]:
df_4_scaled['VWAP_signal'] = df_4_scaled.apply(determine_signal, axis=1)

In [149]:
#retrieve all columns with the substring '_signal'
signal_cols = [col for col in df_4_scaled.columns if col.lower().endswith('_signal')]
sentinment_cols = [col for col in df_4_scaled.columns if col.lower().endswith('_sentiment_weibull_normalized')]

In [150]:
signal_cols

['EMA_Signal',
 'MACD_Signal',
 'BB_RSI_Signal',
 'RSI_Signal',
 'GoldenCross_Signal',
 'Stochastic_Signal',
 'OBV_Signal',
 'VWAP_signal']

In [151]:
sentinment_cols

['reddit_smoothed_sentiment_weibull_normalized',
 'news_smoothed_sentiment_weibull_normalized',
 'tele_smoothed_sentiment_weibull_normalized']

In [152]:
df_4_scaled[sentinment_cols].describe()

,reddit_smoothed_sentiment_weibull_normalized,news_smoothed_sentiment_weibull_normalized,tele_smoothed_sentiment_weibull_normalized
count,7.930000e+03,7930.000000,7930.000000
mean,-6.231418e-03,0.039179,0.035783
std,1.827240e-01,0.172568,0.109249
min,-1.000000e+00,-1.000000,-1.000000
25%,-7.139395e-02,-0.050110,0.000000
50%,-2.200541e-18,-0.013757,0.000000
75%,5.621021e-02,0.094995,0.000000
max,1.000000e+00,1.000000,1.000000


In [153]:
combined_strategy_cols = signal_cols + sentinment_cols

In [154]:
strategy_dict = {col: 0 for col in combined_strategy_cols}

In [155]:
strategy_dict

{'EMA_Signal': 0,
 'MACD_Signal': 0,
 'BB_RSI_Signal': 0,
 'RSI_Signal': 0,
 'GoldenCross_Signal': 0,
 'Stochastic_Signal': 0,
 'OBV_Signal': 0,
 'VWAP_signal': 0,
 'reddit_smoothed_sentiment_weibull_normalized': 0,
 'news_smoothed_sentiment_weibull_normalized': 0,
 'tele_smoothed_sentiment_weibull_normalized': 0}

In [156]:
for i in range(len(strategy_dict.keys())):
    strategy_dict[list(strategy_dict.keys())[i]] = 1/len(strategy_dict.keys())

In [157]:
strategy_dict

{'EMA_Signal': 0.09090909090909091,
 'MACD_Signal': 0.09090909090909091,
 'BB_RSI_Signal': 0.09090909090909091,
 'RSI_Signal': 0.09090909090909091,
 'GoldenCross_Signal': 0.09090909090909091,
 'Stochastic_Signal': 0.09090909090909091,
 'OBV_Signal': 0.09090909090909091,
 'VWAP_signal': 0.09090909090909091,
 'reddit_smoothed_sentiment_weibull_normalized': 0.09090909090909091,
 'news_smoothed_sentiment_weibull_normalized': 0.09090909090909091,
 'tele_smoothed_sentiment_weibull_normalized': 0.09090909090909091}

In [158]:
df_4_scaled['datetime'] = pd.to_datetime(df_4_scaled['datetime'])

In [159]:
df_filtered =df_4_scaled.dropna().set_index('datetime')

In [160]:
df_filtered

,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,...,H-L,H-PC,L-PC,TR,ATR,reddit_smoothed_sentiment_weibull_normalized,news_smoothed_sentiment_weibull_normalized,tele_smoothed_sentiment_weibull_normalized,VWAP,VWAP_signal
datetime,,,,,,,,,,,,,,,,,,,,,
2024-03-02 03:00:00+00:00,0.092267,-204.409405,0.552411,62111.18,62426.70,62025.00,62307.45,67185,27772,39413,...,401.70,315.52,86.18,401.70,347.451667,0.017330,-0.039855,0.069434,62221.006727,1.0
2024-03-02 04:00:00+00:00,0.050600,-205.344729,0.000000,62307.45,62307.45,62035.95,62035.95,56098,24794,31304,...,271.50,0.00,271.50,271.50,333.268333,0.009504,-0.040038,0.000000,62164.619352,0.0
2024-03-02 05:00:00+00:00,0.026060,-200.706776,0.000000,62035.95,62079.99,61752.30,62006.55,65329,23953,41376,...,327.69,44.04,283.65,327.69,326.113333,0.004895,-0.039133,0.000000,62121.439324,0.0
2024-03-02 06:00:00+00:00,0.012619,-185.091972,1.010990,62006.55,62256.00,61930.00,61946.00,57826,25109,32717,...,326.00,249.45,76.55,326.00,325.211667,0.002370,-0.036089,0.127074,62091.659027,0.0
2024-03-02 07:00:00+00:00,0.005751,-164.865815,0.000000,61946.00,62041.70,61790.98,62011.85,86425,28152,58273,...,250.72,95.70,155.02,250.72,307.548333,0.001080,-0.032145,0.000000,62078.579489,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,22248,19527,...,150.00,109.25,40.75,150.00,298.715000,0.010389,-0.002645,0.000000,72443.993188,1.0
2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,30294,32987,...,358.05,223.01,135.04,358.05,325.368333,0.005710,-0.001322,0.012976,72444.833702,1.0
2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,41647,49456,...,586.40,0.01,586.39,586.40,326.935000,0.002934,0.000651,0.000000,72446.652735,1.0


In [161]:
df_filtered = df_filtered.rename(columns={'OPEN':'Open',
                                          'CLOSE':'Close',
                                          'HIGH':'High',
                                          'LOW':'Low'})

In [162]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7929 entries, 2024-03-02 03:00:00+00:00 to 2025-01-26 11:00:00+00:00
Data columns (total 52 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   reddit_smoothed_sentiment_weibull             7929 non-null   float64
 1   news_smoothed_sentiment_weibull               7929 non-null   float64
 2   tele_smoothed_sentiment_weibull               7929 non-null   float64
 3   Open                                          7929 non-null   float64
 4   High                                          7929 non-null   float64
 5   Low                                           7929 non-null   float64
 6   Close                                         7929 non-null   float64
 7   TOTAL_TRADES                                  7929 non-null   int64  
 8   TOTAL_TRADES_BUY                              7929 non-null   int64  
 9   TOTAL_TRADES_SE

In [163]:
class MultiStrategyAggregator(Strategy):
    def __init__(self,weights):
        super().__init__()
        self.weights = weights # as a dictionary of name and weight

    def on_bar(self):

        weights_keys = list(self.weights.keys())

        #map the features to the weights and sum score * weight
        features = self.data.loc[self.current_idx][weights_keys]

        pred_score = sum([features[key] * self.weights[key] for key in weights_keys if features[key] >= 0])

        print(pred_score)

        if pred_score > 0.3:
            signal_strength = pred_score - 0.2
            spot = self.data.loc[self.current_idx]['Close']
            vol = self.data.loc[self.current_idx]['hourly_volatility']

#            print(spot,vol)

#            print(round(0.10 * (self.cash + (spot * self.position_size())),8))

            self.buy(
                "btc",
                size = round(0.10 * (self.cash + (spot * self.position_size())) / spot,8),
            )

In [169]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7929 entries, 2024-03-02 03:00:00+00:00 to 2025-01-26 11:00:00+00:00
Data columns (total 52 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   reddit_smoothed_sentiment_weibull             7929 non-null   float64
 1   news_smoothed_sentiment_weibull               7929 non-null   float64
 2   tele_smoothed_sentiment_weibull               7929 non-null   float64
 3   Open                                          7929 non-null   float64
 4   High                                          7929 non-null   float64
 5   Low                                           7929 non-null   float64
 6   Close                                         7929 non-null   float64
 7   TOTAL_TRADES                                  7929 non-null   int64  
 8   TOTAL_TRADES_BUY                              7929 non-null   int64  
 9   TOTAL_TRADES_SE

In [170]:
e = Engine()
e.add_data(df_filtered)
e.add_strategy(MultiStrategyAggregator(strategy_dict)) # Pass both df_filtered and strategy_dict
stats = e.run()

  0%|          | 0/7929 [00:00<?, ?it/s]

0.18970586421364835
0.09177310004556222
0.09135406827476757
0.011767679356660047
0.09100729226046811
4.2150019350310885e-05
1.7048851486482648e-05
0.1818246833034619
2.3383879736125884e-06
0.2727280662241788
2.541058276343405e-07
0.09090916772005642
0.005962379889915161
0.1818181877254168
0.0023925187520409995
0.18181818217957346
0.10994824325544986
0.03144601552545647
0.09090909091265915
6.833834432846122e-13
0.2448008322673098
2.115228007744847e-14
0.14484713260779541
5.223049935914326e-16
0.04386019084996217
0.1886012093901242
0.008342490424357423
0.040557433695181076
0.007326797044259236
0.02352878125345947
0.09479216521623811
0.023557649238874052
0.09106163624165148
0.18181818181818182
0.18181818181818182
0.18181818181818182
0.29374952930654136
0.18594738047232945
0.19322857550224878
0.28589666672166564
0.19494506398540948
0.30099066891525006
2024-03-03 21:00:00+00:00 New position opened: <Position: btc size: 0.15934226 entry: 62757.99>
0.20758974281425802
2024-03-03 22:00:00+00:0

  1%|▏         | 103/7929 [00:00<00:07, 1015.37it/s]

2024-03-05 21:00:00+00:00 New position opened: <Position: btc size: 0.1615866 entry: 61886.32>
0.09090909090909091
0.0
2024-03-05 23:00:00+00:00 Take Profit Filled. Limit: 64319.25 / High: 64367.27
2024-03-05 23:00:00+00:00 Position closed for ticker btc at 64319.25
0.05724312734409379
0.0
0.0
0.0
0.09090909090909091
0.0
0.18181818181818182
0.10862689460760108
0.2727272727272727
0.18181818181818182
0.1959164446720304
0.2727272727272727
0.19972345156690646
0.20178743429491108
0.2727272727272727
0.205254331863508
0.30460695075426253
2024-03-06 16:00:00+00:00 New position opened: <Position: btc size: 0.15041865 entry: 66481.12>
0.1934013086267433
0.20204836331491546
0.2522339067188528
0.22262638264196288
0.31832307536980275
2024-03-06 21:00:00+00:00 New position opened: <Position: btc size: 0.16454498 entry: 66888.24>
0.22728518734641479
2024-03-06 22:00:00+00:00 Stop Loss Filled. Limit: 66159.89333333334 / Low: 65896.25
2024-03-06 22:00:00+00:00 Position closed for ticker btc at 66159.89

  3%|▎         | 205/7929 [00:00<00:08, 933.80it/s] 

...

0.09090909090909091
0.18181818181818182
0.10295335797888472
0.2727272727272727
0.18232861798187294
0.18504143650383786
0.2828802730741434
0.19064215527212236
0.09713102302085885
0.09409354866395217
0.09130903257023279
0.2727272727272727
0.09420189408540793
0.2727272727272727
0.09090909090909091
0.2735086109670712
0.09533434824542797
0.09444375402774653
0.09492284811992993
0.09910883726312701
0.11697866098348067
0.1993780898103178
0.11084983565908299
0.13024414586796923
0.20052921945347793
0.10548136085073714
0.1900639390182254
0.09884679533368744
0.18620582852877549
0.09090909090909091
0.18181818181818182
0.09090909090909091
0.18181818181818182
0.18181818181818182
0.18181818181818182
0.2727272727272727
0.19391953697075776
0.18181818181818182
0.18181818181818182
0.18181818181818182
0.18181818181818182
0.18181818181818182
0.1924272423092903
0.18181818181818182
0.2727272727272727
0.19660025170631631
0.2727272727272727
0.09090909090909091
0.18181818181818182
0.11097100546215016
0.181818181

 97%|█████████▋| 7659/7929 [00:06<00:00, 1129.97it/s]

0.3025932502473891
2025-01-05 03:00:00+00:00 New position opened: <Position: btc size: 0.10180451 entry: 98227.48>
0.21718857604224864
2025-01-05 04:00:00+00:00 Take Profit Filled. Limit: 98441.68166666667 / High: 98522.76
2025-01-05 04:00:00+00:00 Position closed for ticker btc at 98441.68166666667
0.2763454427583501
0.18181818181818182
0.2727272727272727
0.20363408302297484
0.18181818181818182
0.09090909090909091
0.2729171430572505
0.09090909090909091
0.36363636363636365
2025-01-05 13:00:00+00:00 New position opened: <Position: btc size: 0.10228085 entry: 97770.01>
0.09434914739861402
2025-01-05 14:00:00+00:00 Stop Loss Filled. Limit: 97452.30833333332 / Low: 97276.79
2025-01-05 14:00:00+00:00 Position closed for ticker btc at 97452.30833333332
0.18181818181818182
0.09090909090909091
0.09090909090909091
0.09090909090909091
0.1826290526598213
0.09582131074925888
0.28242597812934983
0.18181818181818182
0.19666967461683285
0.19595487276725934
0.19301192642378318
0.28009147027860765
0.18

100%|██████████| 7929/7929 [00:06<00:00, 1147.76it/s]


0.19644492414381054
0.37922777708741423
2025-01-20 02:00:00+00:00 New position opened: <Position: btc size: 0.09931195 entry: 100692.82>
0.10707374715871142
0.10656038452927113
0.19533262336780044
2025-01-20 05:00:00+00:00 Take Profit Filled. Limit: 102533.90833333334 / High: 102825.73
2025-01-20 05:00:00+00:00 Position closed for ticker btc at 102533.90833333334
0.10140798651462472
0.18718129344690593
0.18217038842732636
0.2727272727272727
0.19510927380755824
0.18181818181818182
0.2727272727272727
0.18181818181818182
0.2746245663594597
0.2155311025792068
0.19209917734722184
0.3115309316839056
2025-01-20 17:00:00+00:00 New position opened: <Position: btc size: 0.09374674 entry: 106670.38>
0.19821357104157497
2025-01-20 18:00:00+00:00 Stop Loss Filled. Limit: 103528.58 / Low: 102738.23
2025-01-20 18:00:00+00:00 Position closed for ticker btc at 103528.58
0.19843435610065707
0.10982657275039545
0.20186341483601503
0.14273775511462658
0.19618112052044637
0.09943292418569938
0.094703961450